In [2]:
import numpy as np
import datetime

The board will be represented by a 3x3 matrix, with components -1, 0 or 1. -1(1) corresponds to player -1(1), and 0 to an empty square. The board shall be saved in the self.board at all times.

A state is the same as the board, with a main difference: -1(1) corresponds to the mark of the current(opponent) player.

A play is a 3x3 matrix with all zeros except one component.

In [18]:
class Board(object):
    def start(self):
        self.board = np.zeros(3, 3)
        return self.board.flatten()

    def current_player(self, state):
        """
        Gets the current player number ::cplayNum::
        """
        whos_turn = np.sum(np.abs(state))
        if whos_turn%2 == 0:
            cplayNum = -1
        else:
            cplayNum = 1
        return cplayNum

    def next_state(self, state, play):
        """
        Takes the game state, and the move to be applied, returns the new game state.
        """
        new_state = state + play
        return new_state

    def legal_plays(self, state):
        """
        Takes the game state and returns the possible legal plays
        """
        idx = np.where(state == 0)[0]
        empty_play = np.zeros(9)
        new_plays = []
        for i in idx:
            copy = empty_play[:]
            copy[i] = -1
            new_plays.append(copy)
        return new_plays

    def winner(self, state):
        matrix = state.reshape(3,3)
        if np.any(matrix.sum(axis=0) == 3) or np.any(np.trace(matrix) == 3) or np.any(np.trace(np.fliplr(matrix)) == 3):
            # player 1 wins
            winner = 1
        elif np.any(matrix.sum(axis=0) == -3) or np.any(np.trace(matrix) == -3) or np.any(np.trace(np.fliplr(matrix)) == -3):
            # player -1 wins
            winner = -1
        elif np.where(state == 0)[0]:
            # game still ongoing
            winner = 0
        else:
            # game is a draw
            winner = 2
        return winner
    
    def stringRepresentation(self, state):
        return ''.join([str(x) for x in state])

The Monte Carlo part follows this convention:

The Deep Boltzmann Machine (DBM)  takes states of the board $s$, by doing $f_\theta:s\mapsto (v_\theta(s), \vec{p}_\theta(s))$, and outputs the board evaluation $v_\theta(s)\in[-1,1]$ (this may need to be adjusted in the DBM class that Juan Florez is writing), and a move policy $\vec{p}_\theta(s)$.

When training the DBM, for each game we give it data of the form $(s_t, \vec{\pi}_t, z_t)$ for all states $s_t$ indexed by $t$. $\vec{\pi}_t$ is an estimate of the move policy from state $s_t$, and $z_t=-1,0,1$ is the outcome of the game, as seen by the player to play at time $t$. Therefore the DBM loss function is
$$
l = \sum_t \left[\left(v_\theta(s_t)-z_t\right)^2-\vec{\pi}_t\cdot \log(\vec{p}_\theta(s_t))\right]
$$

Let $Q(s,a)$ be the expected reward for making play $a$ from state $s$; $N(s,a)$ the number of times $a$ was played from $s$ across all simulations; $P(s,a)$ the probability that $a$ is played from $s$ according to the DBM. Therefore, the confidence upper bound is

$$
U(s,a) = Q(s,a) + c P(s,a) \frac{\sqrt{\sum_b N(s,b)}}{N(s,a)+1},
$$

where $c$ is a constant that tunes the degree of exploration within the tree of moves.

In [6]:
import math
import numpy as np
EPS = 1e-8

class MCTS():
    """
    This class handles the MCTS tree.
    """

    def __init__(self, board, nnet, num_sims):
        self.board = board
        self.nnet = nnet
        self.num_sims = num_sims
        self.Qsa = {}       # stores Q values for s,a (as defined in the paper)
        self.Nsa = {}       # stores #times edge s,a was visited
        self.Ns = {}        # stores #times board s was visited
        self.Ps = {}        # stores initial policy (returned by neural net)

        self.Es = {}        # stores game.getGameEnded ended for board s
        self.Vs = {}        # stores game.getValidMoves for board s

    def getActionProb(self, state, temp=1):
        """
        This function performs num_sims simulations of MCTS starting from
        state.
        Returns:
            probs: a policy vector where the probability of the ith action is
                   proportional to Nsa[(s,a)]**(1./temp)
        """
        for i in range(self.num_sims):
            self.search(state)

        s = self.board.stringRepresentation(state)
        counts = [self.Nsa[(s,a)] if (s,a) in self.Nsa else 0 for a in 
                  [self.board.stringRepresentation(x) for x in self.game.legal_plays(state)]]

        if temp==0:
            bestA = np.argmax(counts)
            probs = [0]*len(counts)
            probs[bestA]=1
            return probs

        counts = [x**(1./temp) for x in counts]
        probs = [x/float(sum(counts)) for x in counts]
        return probs


    def search(self, state):
        """
        This function performs one iteration of MCTS. It is recursively called
        till a leaf node is found. The action chosen at each node is one that
        has the maximum upper confidence bound as in the paper.
        Once a leaf node is found, the neural network is called to return an
        initial policy P and a value v for the state. This value is propogated
        up the search path. In case the leaf node is a terminal state, the
        outcome is propogated up the search path. The values of Ns, Nsa, Qsa are
        updated.
        NOTE: the return values are the negative of the value of the current
        state. This is done since v is in [-1,1] and if v is the value of a
        state for the current player, then its value is -v for the other player.
        Returns:
            v: the negative of the value of the current state
        """

        s = self.game.stringRepresentation(state)

        if s not in self.Es:
            self.Es[s] = self.game.getGameEnded(state, 1)
        if self.Es[s]!=0:
            # terminal node
            return -self.Es[s]

        if s not in self.Ps:
            # leaf node
            self.Ps[s], v = self.nnet.predict(state)
            valids = self.game.getValidMoves(state, 1)
            self.Ps[s] = self.Ps[s]*valids      # masking invalid moves
            sum_Ps_s = np.sum(self.Ps[s])
            if sum_Ps_s > 0:
                self.Ps[s] /= sum_Ps_s    # renormalize
            else:
                # if all valid moves were masked make all valid moves equally probable
                
                # NB! All valid moves may be masked if either your NNet architecture is insufficient or you've get overfitting or something else.
                # If you have got dozens or hundreds of these messages you should pay attention to your NNet and/or training process.   
                print("All valid moves were masked, do workaround.")
                self.Ps[s] = self.Ps[s] + valids
                self.Ps[s] /= np.sum(self.Ps[s])

            self.Vs[s] = valids
            self.Ns[s] = 0
            return -v

        valids = self.Vs[s]
        cur_best = -float('inf')
        best_act = -1

        # pick the action with the highest upper confidence bound
        for a in range(self.game.getActionSize()):
            if valids[a]:
                if (s,a) in self.Qsa:
                    u = self.Qsa[(s,a)] + self.args.cpuct*self.Ps[s][a]*math.sqrt(self.Ns[s])/(1+self.Nsa[(s,a)])
                else:
                    u = self.args.cpuct*self.Ps[s][a]*math.sqrt(self.Ns[s] + EPS)     # Q = 0 ?

                if u > cur_best:
                    cur_best = u
                    best_act = a

        a = best_act
        next_s, next_player = self.game.getNextState(state, 1, a)
        next_s = self.game.getCanonicalForm(next_s, next_player)

        v = self.search(next_s)

        if (s,a) in self.Qsa:
            self.Qsa[(s,a)] = (self.Nsa[(s,a)]*self.Qsa[(s,a)] + v)/(self.Nsa[(s,a)]+1)
            self.Nsa[(s,a)] += 1

        else:
            self.Qsa[(s,a)] = v
            self.Nsa[(s,a)] = 1

        self.Ns[s] += 1
        return -v

SyntaxError: invalid syntax (<ipython-input-6-416b416ae87f>, line 37)